# Examples of Pyspark ML

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [5]:
spark

In [7]:
## Read the dataset
training=spark.read.csv('pyspark_test_data_5.csv',header=True,inferSchema=True)

In [8]:
training.show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|Shayema| 22|        10| 30000|
|Susmita| 30|         8| 25000|
|  Sonam| 29|         4| 20000|
|  Nidhi| 24|         3| 20000|
|   Puja| 21|         2| 15000|
|Aliviya| 23|         1| 18000|
+-------+---+----------+------+



In [9]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [15]:
##  [age,Experience]--->new feature----->independent feature

from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Feature")

In [16]:
output=featureassembler.transform(training)

In [17]:
output.show()

+-------+---+----------+------+-------------------+
|   Name|age|Experience|Salary|Independent Feature|
+-------+---+----------+------+-------------------+
|Shayema| 22|        10| 30000|        [22.0,10.0]|
|Susmita| 30|         8| 25000|         [30.0,8.0]|
|  Sonam| 29|         4| 20000|         [29.0,4.0]|
|  Nidhi| 24|         3| 20000|         [24.0,3.0]|
|   Puja| 21|         2| 15000|         [21.0,2.0]|
|Aliviya| 23|         1| 18000|         [23.0,1.0]|
+-------+---+----------+------+-------------------+



In [18]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [19]:
finalized_data=output.select("Independent Feature","Salary")

In [20]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [22.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,2.0]| 15000|
|         [23.0,1.0]| 18000|
+-------------------+------+



In [21]:
from pyspark.ml.regression import LinearRegression
## train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Feature",labelCol="Salary")
regressor=regressor.fit(train_data)

In [23]:
## Coefficients
regressor.coefficients

DenseVector([-7.8003, 1751.17])

In [24]:
## Intercepts
regressor.intercept

12191.887675507007

In [25]:
## Prediction
pred_results=regressor.evaluate(test_data)

In [27]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [23.0,1.0]| 18000|13763.650546021836|
|         [24.0,3.0]| 20000|  17258.1903276131|
+-------------------+------+------------------+



In [28]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(3489.0795631825313, 12732088.487907723)